# Prognoza cen mieszkań w Bostonie

## Problem

Jaką cenę będzie miało mieszkanie w zadanej okolicy?

## Zbiór nauczający

In [2]:
import pandas as pd
import numpy as np
zbior_nauczajacy = pd.read_csv("Boston.csv", index_col=False)

zbior_nauczajacy.head(3)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,16.81180,0.0,18.1,0,0.700,5.277,98.1,1.4261,24,666,20.2,396.90,30.81,7.2
1,6.71772,0.0,18.1,0,0.713,6.749,92.6,2.3236,24,666,20.2,0.32,17.44,13.4
2,12.80230,0.0,18.1,0,0.740,5.854,96.6,1.8956,24,666,20.2,240.52,23.79,10.8


Cechy:
 
CRIM - per capita crime rate by town   
ZN - proportion of residential land zoned for lots over 25,000 sq.ft.   
INDUS - proportion of non-retail business acres per town.   
CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)   
NOX - nitric oxides concentration (parts per 10 million)   
RM - average number of rooms per dwelling   
AGE - proportion of owner-occupied units built prior to 1940   
DIS - weighted distances to five Boston employment centres   
RAD - index of accessibility to radial highways   
TAX - full-value property-tax rate per $10,000   
PTRATIO - pupil-teacher ratio by town   
BLACK - where Bk is the proportion of blacks by town   
LSTAT - lower status of the population   


Cecha, którą chcemy prognozować:
  
MEDV - Median value of owner-occupied homes in $1000's   

### Zbiór treningowy i testowy

Na wstępie, podzielmy losowo nasz zbiór na dwie części: 

1. zbior_treningowy: na tych danych będziemy nauczali modele. Przyjmijmy, że będzie to 80% całego posiadanego zbioru.
2. zbior_testowy: na tych danych będziemy testowali nasze modele. Będzie to 20% całego naszego zbioru danych

In [3]:
from sklearn.model_selection import train_test_split

zbior_treningowy, zbior_testowy = train_test_split(zbior_nauczajacy, test_size=0.2)

#sprawdźmy wymiary
zbior_treningowy.shape, zbior_testowy.shape

((384, 14), (96, 14))

## Model 1

Zaczniemy od wyboru cech, na których będziemy nauczali nasz model. Przyjmijmy, że będą to wszystkie cechy:  
Od: CRIM - per capita crime rate by town  
Do: LSTAT - lower status of the population 

Zwyczajowo, odpowiednie zbiory danych nazywa się:  
X_train: zbiór TRENINGOWY z cechami objaśniającymi  
y_train: zbiór TRENINGOWY z cechą, której chcemy nauczyć się odgadywać.  


In [4]:
cechy_objasniajace = [
    'crim',
    'zn',
    'indus',
    'chas',
    'nox',
    'rm',
    'age',
    'dis',
    'rad',
    'tax',
    'ptratio',
    'black',
    'lstat'
    ]

In [5]:
X_train = zbior_treningowy.loc[:, cechy_objasniajace] # TRENINGOWY zbiór danych z cechami objaśniającymi
y_train = zbior_treningowy['medv'] # TRENINGOWY zbiór danych z cechą, której chcemy nauczyć się odgadywać.

In [6]:
X_train.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
118,0.04684,0.0,3.41,0,0.4890,6.417,66.1,3.0923,2,270,17.8,392.18,8.81
28,0.13960,0.0,8.56,0,0.5200,6.167,90.0,2.4210,5,384,20.9,392.69,12.33
378,0.05660,0.0,3.41,0,0.4890,7.007,86.3,3.4217,2,270,17.8,396.90,5.50
210,0.03510,95.0,2.68,0,0.4161,7.853,33.2,5.1180,4,224,14.7,392.78,3.81
176,0.06263,0.0,11.93,0,0.5730,6.593,69.1,2.4786,1,273,21.0,391.99,9.67


In [7]:
type(y_train)

pandas.core.series.Series

A teraz zrobimy to samo, ale dla zbioru, na którym będziemy sprawdzać poprawność naszego modelu.

Zwyczajowo, odpowiednie zbiory danych nazywa się:  
X_test: zbiór TESTOWY z cechami objaśniającymi  
y_test: zbiór TESTOWY z cechą, której chcemy nauczyć się odgadywać.  

In [8]:
X_test = zbior_testowy.loc[:, 'crim':'lstat'] # TESTOWY zbiór danych z cechami objaśniającymi
y_test= zbior_testowy['medv'] # TESTOWY zbiór danych z cechą, której chcemy nauczyć się odgadywać.

Sprawdźmy jeszcze liczbę rekordów w poszczególnych plikach...

In [9]:
print("Będziemy mieli %d rekordów treningowych oraz %d rekordów testowych." % (len(X_train), len(X_test)))

Będziemy mieli 384 rekordów treningowych oraz 96 rekordów testowych.


### Model

Zacznijmy od zainicjowania modelu. Będzie to nasz prototyp, jeszcze nie nauczony...

In [10]:
from sklearn.linear_model import LinearRegression
model_początkowy_1 = LinearRegression()

### Trening

Wytrenujmy nasz model początkowy na danych treningowych ("dopasujmy" (fit) do tych danych)

In [11]:
model_nauczony_1 = model_początkowy_1.fit(X_train,y_train)

### Ewaluacja

Teraz sprawdzimy, na ile dobrze prognozuje nasz model.   


W pierwszej kolejności poprosimy nasz model o predykcję dla zbioru testowego.  
Wygląda on następująco: jak widać, nie zawiera kolumny ze zmienną prognozowaną.

In [12]:
X_test.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
372,0.06664,0.0,4.05,0,0.510,6.546,33.1,3.1323,5,296,16.6,390.96,5.33
115,37.66190,0.0,18.10,0,0.679,6.202,78.7,1.8629,24,666,20.2,18.82,14.52
131,0.02177,82.5,2.03,0,0.415,7.610,15.7,6.2700,2,348,14.7,395.38,3.11
387,0.13554,12.5,6.07,0,0.409,5.594,36.8,6.4980,4,345,18.9,396.90,13.09
186,0.37578,0.0,10.59,1,0.489,5.404,88.6,3.6650,4,277,18.6,395.24,23.98


Poprośmy teraz nasz model o jego prognozy

In [13]:
type(model_nauczony_1)

sklearn.linear_model._base.LinearRegression

In [14]:
prognozy = model_nauczony_1.predict(X_test)

Obejrzyjmy zbiór z prognozami...

In [15]:
type(prognozy)

numpy.ndarray

In [16]:
print(prognozy)
print("\n")
print("Zbiór z prognozami ma %d wartości, zaś zbiór testowy ma ich %d." % (len(prognozy), len(X_test)))

[30.53687828 14.06238477 36.4172301  17.45188651 17.16695532 33.02662817
 19.41304493 12.08117153 31.41012964 30.80571351 13.5339028  10.39069708
 15.61626837 38.73244833 20.40008918 16.93511004 23.51867361 25.47473644
 24.15385363 35.78551062 35.21186218 34.16750417 32.25753437 24.0999721
 19.97234824 15.76133319  7.03742973 10.60655873 34.0687733  24.79834744
 16.1389775  20.81314201 29.93797647 22.21032628  8.46846115 22.42934065
 23.34150473 27.43741338 32.71775049 17.40475721 20.9987836  21.91083823
 20.22061967 19.36037292 30.5387472  40.01515574 16.88304646 15.21774396
 14.2283461  24.21369817 21.90014901 11.73704495 14.60265785 25.18565368
 18.92114282 23.91418803 15.49463493 24.0834206  30.99442253 20.36217527
 17.19941471 12.79753593 25.19735813 35.91735238 20.68619295 17.76787961
 26.50897193 25.99949637 24.70595137 17.72618066 34.38675085 30.01072579
 20.32749028 17.17790869 21.22951123 23.77048377 19.92023154 14.5406666
 19.07957976 36.32551439 23.13933457 13.92413782 27.0

Porównajmy teraz prognozy modelu z prawdziwymi danymi ze zbioru y_test.
Zaczniemy od porównania parami...


In [17]:
for i in range(len(prognozy)):
    print(prognozy[i], y_test.to_numpy()[i])

30.536878275781362 29.4
14.062384765772332 10.9
36.417230104035276 42.3
17.451886512535282 17.4
17.166955315615663 19.3
33.02662816846513 41.3
19.41304493257869 18.5
12.081171528097329 15.6
31.410129637128268 29.9
30.80571350798985 35.4
13.533902799927475 12.3
10.390697081254999 23.7
15.616268373595695 16.6
38.73244833315113 50.0
20.400089177443057 20.4
16.935110036067876 10.2
23.51867360753187 19.4
25.474736435220972 30.1
24.15385363240855 22.2
35.785510615617575 32.4
35.211862175567006 35.1
34.167504167051575 39.8
32.25753436976568 31.6
24.09997210314981 18.9
19.972348237200343 13.3
15.761333192897226 15.6
7.037429726948801 10.5
10.606558733632745 9.7
34.06877330303369 43.8
24.798347442823726 24.1
16.138977496338068 20.2
20.813142008380048 18.7
29.937976470951085 30.1
22.210326283313655 20.3
8.46846114803482 7.2
22.429340654334105 23.2
23.341504733444204 24.4
27.437413379383035 36.2
32.71775049031551 27.0
17.404757212370214 23.2
20.99878359618304 18.8
21.91083823195418 17.8
20.220619

Na koniec, obliczmy łączną miarę dokładności predykcji: RMSE (Root Mean Square Error).

In [18]:
from sklearn.metrics import mean_squared_error
rmse_1 = np.sqrt(mean_squared_error(y_test, prognozy))
print('RMSE = ', rmse_1)

RMSE =  5.0032750438919615


Na ile jest to dokładne? Obliczmy rozpiętość wartości skrajnych cen.

In [19]:
print('Wartość minimalna w zbiorze testowym: %d' %(min(y_test)))
print('Wartość maksymalna w zbiorze testowym: %d' %(max(y_test)))

Wartość minimalna w zbiorze testowym: 7
Wartość maksymalna w zbiorze testowym: 50


## Model 2

Wybierz cechy, na których będziesz nauczał(a) model.


In [20]:
cechy_objasniajace = [
    'crim',
    'zn',
    #'indus',
    #'chas',
    #'nox',
    #'rm',
    #'age',
    #'dis',
    #'rad',
    #'tax',
    #'ptratio',
    #'black',
    'lstat'
    ]

In [21]:
X_train = zbior_treningowy.loc[:, cechy_objasniajace] # TRENINGOWY zbiór danych z cechami objaśniającymi
y_train = zbior_treningowy['medv'] # TRENINGOWY zbiór danych z cechą, której chcemy nauczyć się odgadywać.

In [22]:
X_train.head()

,crim,zn,lstat
118,0.04684,0.0,8.81
28,0.13960,0.0,12.33
378,0.05660,0.0,5.50
210,0.03510,95.0,3.81
176,0.06263,0.0,9.67


Wybierz odpowiednie cechy w zbiorze testowym 

In [23]:
X_test = zbior_testowy.loc[:, cechy_objasniajace] # TESTOWY zbiór danych z cechami objaśniającymi
y_test= zbior_testowy['medv'] # TESTOWY zbiór danych z cechą, której chcemy nauczyć się odgadywać.

Sprawdźmy jeszcze liczbę rekordów w poszczególnych plikach...

### Model

Zacznijmy od zainicjowania modelu. Będzie to nasz prototyp, jeszcze nie nauczony...

In [24]:
from sklearn.linear_model import LinearRegression
model_początkowy_2 = LinearRegression()


### Trening

Wytrenujmy nasz model początkowy na danych treningowych ("dopasujmy" (fit) do tych danych)

In [25]:
model_nauczony_2 = model_początkowy_2.fit(X_train,y_train)

### Ewaluacja

Teraz sprawdzimy, na ile dobrze prognozuje nasz model.   


In [26]:
prognozy = model_nauczony_2.predict(X_test)

Porównajmy teraz prognozy modelu z prawdziwymi danymi ze zbioru y_test.
Obliczmy łączną miarę dokładności predykcji: RMSE (Root Mean Square Error).

In [27]:
from sklearn.metrics import mean_squared_error
rmse_2 = np.sqrt(mean_squared_error(y_test, prognozy))
print('RMSE_2 = ', rmse_2)

RMSE_2 =  6.653298520278653


## Model 3

Tym razem, poniżej zobaczysz metodę projektowania i tworzenia modelu w najbardziej syntetycznej formie, często spotykanej w realnych zastosowaniach.

In [28]:
# Zaimportuj niezbędne biblioteki

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Wybierz cechy, na których będziesz nauczał(a) model.
cechy_objasniajace = [
    'crim',
    'zn',
    'indus',
    'chas',
    'nox',
    'rm',
    'age',
    'dis',
    'rad',
    'tax',
    'ptratio',
    'black',
    'lstat'
    ]
# Stwórz zbiory: treningowy i testowy
X_train = zbior_treningowy.loc[:, cechy_objasniajace] # TRENINGOWY zbiór danych z cechami objaśniającymi
y_train = zbior_treningowy['medv'] # TRENINGOWY zbiór danych z cechą, której chcemy nauczyć się odgadywać.

X_test = zbior_testowy.loc[:, cechy_objasniajace] # TESTOWY zbiór danych z cechami objaśniającymi
y_test= zbior_testowy['medv'] # TESTOWY zbiór danych z cechą, której chcemy nauczyć się odgadywać.

# Zainicjuj model
model_początkowy_3 = Ridge()

# Wytrenuj go
model_nauczony_3 = model_początkowy_3.fit(X_train,y_train)

# Sprawdź jego efektywność
prognozy = model_nauczony_3.predict(X_test) #wygeneruj prognozy

# Oblicz miarę efektywności
rmse_3 = np.sqrt(mean_squared_error(y_test, prognozy))
print('RMSE_3 = ', rmse_3)


RMSE_3 =  5.006179826722167


## Wnioski

Porównajmy teraz wyniki poszczególnych modeli.

In [29]:
modele = [model_nauczony_1, model_nauczony_2, model_nauczony_3]
for i in range(3):
    print(modele[i])

LinearRegression()
LinearRegression()
Ridge()


In [30]:
RMSE = [rmse_1, rmse_2, rmse_3]
for i in range(3):
    print('RMSE_',i+1, ' = ', RMSE[i])

RMSE_ 1  =  5.0032750438919615
RMSE_ 2  =  6.653298520278653
RMSE_ 3  =  5.006179826722167


## Eksport modelu do pliku

Najlepszym modelem jest model model_nauczony_1. Zapiszmy go więc do pliku tak, by móc później z niego korzystać w praktyce.

In [31]:
import pickle

pickle.dump(model_nauczony_1, open('nasz_regresor.sav', 'wb'))